# Purpose of this notebook

In [24]:
import xarray as xr, matplotlib.pyplot as plt
#import sys 
#sys.path.append('') # use this if the function file is in a different directory to the notebook
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration

In [25]:
!ls /g/data/ua8/C20C/v3/

LAND.nc     member_daily				     subdaily
mean_3hr    README_20C_Reanalysis_version_3_everymember.txt  tmp
mean_daily  spread_daily


In [26]:
!ls /g/data/ua8/C20C/v3/mean_daily

prate  prmsl  tmax_2m  tmin_2m


In [27]:
!ls /g/data/ua8/C20C/v3/mean_daily/prate

prate.1836.nc  prate.1872.nc  prate.1908.nc  prate.1944.nc  prate.1980.nc
prate.1837.nc  prate.1873.nc  prate.1909.nc  prate.1945.nc  prate.1981.nc
prate.1838.nc  prate.1874.nc  prate.1910.nc  prate.1946.nc  prate.1982.nc
prate.1839.nc  prate.1875.nc  prate.1911.nc  prate.1947.nc  prate.1983.nc
prate.1840.nc  prate.1876.nc  prate.1912.nc  prate.1948.nc  prate.1984.nc
prate.1841.nc  prate.1877.nc  prate.1913.nc  prate.1949.nc  prate.1985.nc
prate.1842.nc  prate.1878.nc  prate.1914.nc  prate.1950.nc  prate.1986.nc
prate.1843.nc  prate.1879.nc  prate.1915.nc  prate.1951.nc  prate.1987.nc
prate.1844.nc  prate.1880.nc  prate.1916.nc  prate.1952.nc  prate.1988.nc
prate.1845.nc  prate.1881.nc  prate.1917.nc  prate.1953.nc  prate.1989.nc
prate.1846.nc  prate.1882.nc  prate.1918.nc  prate.1954.nc  prate.1990.nc
prate.1847.nc  prate.1883.nc  prate.1919.nc  prate.1955.nc  prate.1991.nc
prate.1848.nc  prate.1884.nc  prate.1920.nc  prate.1956.nc  prate.1992.nc
prate.1849.nc  prate.1885.nc  prate.19

In [43]:
!ls /g/data/ua8/C20C/v3/mean_daily/tmax_2m/

tmax.2m.1836.nc  tmax.2m.1881.nc  tmax.2m.1926.nc  tmax.2m.1971.nc
tmax.2m.1837.nc  tmax.2m.1882.nc  tmax.2m.1927.nc  tmax.2m.1972.nc
tmax.2m.1838.nc  tmax.2m.1883.nc  tmax.2m.1928.nc  tmax.2m.1973.nc
tmax.2m.1839.nc  tmax.2m.1884.nc  tmax.2m.1929.nc  tmax.2m.1974.nc
tmax.2m.1840.nc  tmax.2m.1885.nc  tmax.2m.1930.nc  tmax.2m.1975.nc
tmax.2m.1841.nc  tmax.2m.1886.nc  tmax.2m.1931.nc  tmax.2m.1976.nc
tmax.2m.1842.nc  tmax.2m.1887.nc  tmax.2m.1932.nc  tmax.2m.1977.nc
tmax.2m.1843.nc  tmax.2m.1888.nc  tmax.2m.1933.nc  tmax.2m.1978.nc
tmax.2m.1844.nc  tmax.2m.1889.nc  tmax.2m.1934.nc  tmax.2m.1979.nc
tmax.2m.1845.nc  tmax.2m.1890.nc  tmax.2m.1935.nc  tmax.2m.1980.nc
tmax.2m.1846.nc  tmax.2m.1891.nc  tmax.2m.1936.nc  tmax.2m.1981.nc
tmax.2m.1847.nc  tmax.2m.1892.nc  tmax.2m.1937.nc  tmax.2m.1982.nc
tmax.2m.1848.nc  tmax.2m.1893.nc  tmax.2m.1938.nc  tmax.2m.1983.nc
tmax.2m.1849.nc  tmax.2m.1894.nc  tmax.2m.1939.nc  tmax.2m.1984.nc
tmax.2m.1850.nc  tmax.2m.1895.nc  tmax.2m.1940.nc  tmax.2m.198

In [28]:
!ls /g/data/lp01/CMIP6/CMIP/BCC/BCC-ESM1/historical/r1i1p1f1/Amon/tas/gr1.5/v20181214/

tas_Amon_BCC-ESM1_historical_r1i1p1f1_gr1.5_185001-201412.nc


In [29]:
# read in precipitation data
d = xr.open_mfdataset('/g/data/ua8/C20C/v3/mean_daily/prate/*.nc', combine='by_coords', chunks={'time': 12})

In [ ]:
# read in temperature data
d = xr.open_mfdataset('/g/data/ua8/C20C/v3/mean_daily/tmax_2m/*.nc', combine='by_coords', chunks={'time': 12})

In [30]:
d

<xarray.Dataset>
Dimensions:    (lat: 181, lon: 360, nbnds: 2, time: 65744)
Coordinates:
  * lat        (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon        (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * time       (time) datetime64[ns] 1836-01-01 1836-01-02 ... 2015-12-31
Dimensions without coordinates: nbnds
Data variables:
    prate      (time, lat, lon) float32 dask.array<chunksize=(12, 181, 360), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(12, 2), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.2
    title:                     mean Daily NOAA/CIRES/DOE 20th Century Reanaly...
    comments:                  Data are from \nNOAA/CIRES/DOE 20th Century Re...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    license:                   These data are available free of charge under ...
    spatial_resolution:        1.0 degree
    platform:                  Model
    version:                   3si
    history:                   created 2019/03 by Hoop (chunked, deflated non...
    source:                    20CRv3si 2018, Ensemble Kalman Filter, ocean (...
    forcing:                   CO2, Sl, Vl, SST, Oz (CO2 as in Saha et al (20...
    name_of_model_used:        ensda_v451
    creation_date:             2019-03-25T21:26:27Z
    product:                   reanalysis
    observations:              International Surface Pressure Databank versio...
    assimilation_algorithm:    Ensemble Kalman Filter with 4DIAU
    dataset_title:             NOAA/CIRES/DOE 20th Century Reanalysis (V3)
    forcing_note:              Additional information on the external forcing...
    citation1:                 Slivinski, L. C, G. P. Compo, J. S. Whitaker, ...
    References:                https://www.psl.noaa.gov/data/gridded/data.20t...
    creator_name:              NOAA/PSL
    institution:               NOAA Physical Sciences Laboratory & CU/CIRES \...
    contact:                   psl.data@noaa.gov
    citation:                  Compo,G.P. <https://www.psl.noaa.gov/people/gi...

In [32]:
d2 = d.mean(dim=['lat', 'lon', 'time'])
del d2['time_bnds']

In [42]:
d2.prate.values

array(2.9006693e-05, dtype=float32)